In [40]:
# SETUP DEBUG FLEET
# (applies the first time step of actions in the example solution)

# MOVE THIS FILE TO THE BASE DIRECTORY IF YOU WANT TO USE IT
# or find a workaround for the error this throws in the greedy_profit dir
from system_state import SystemState
from utils import (load_problem_data,
                   load_solution)

solution = load_solution('./data/solution_example.json')
demand, datacenters, servers, selling_prices = load_problem_data()

state: SystemState = SystemState(datacenters=datacenters, servers=servers)

first_time_step_actions = solution.set_index('time_step').loc[1].to_dict('records')
state.update_state(first_time_step_actions)

print(state.fleet)

    datacenter_id server_generation                             server_id  \
0             DC1            CPU.S1  7ee8a1de-b4b8-4fce-9bd6-19fdf8c1e409   
1             DC1            CPU.S1  ed7946d9-6b09-4d58-85b7-be5c25437c1d   
2             DC1            CPU.S1  0c2dd066-2628-4935-b540-3ab27a45e824   
3             DC1            CPU.S1  fa4e5408-db94-4a08-92ca-7a81cda8b4c2   
4             DC1            CPU.S1  1841c99d-8f0e-40d9-af28-72e485495d2c   
..            ...               ...                                   ...   
157           DC3            GPU.S1  5ffa6e3d-3d77-4951-8913-71168e307af4   
158           DC3            GPU.S1  06cf58b9-adf0-4711-8812-468f8f4af732   
159           DC4            GPU.S1  b16bbdf4-29bc-4aa4-ac72-6bd3ec5c26c8   
160           DC4            GPU.S1  e8ab2d4f-87ee-41bb-b693-df4b208afcdc   
161           DC4            GPU.S1  d7212382-cf13-4fa3-bdbc-0045d10dd1c8   

     lifespan  life_expectancy latency_sensitivity  capacity  purchase_pric

In [62]:
# TESTING THE GREEDY PROFIT ALGORITHM/CALCULATING SERVERS TO BUY
import math
from evaluation import get_actual_demand, get_capacity_by_server_generation_latency_sensitivity, get_time_step_demand
import numpy as np
import pandas as pd

actual_demand = get_actual_demand(demand)

def get_unsatisfied_demand(actual_demand: pd.DataFrame, fleet: pd.DataFrame, time_step: int):
    current_demand = get_time_step_demand(actual_demand, time_step)
    
    # The fleet is empty, so unsatisfied demand = all demand
    if time_step == 1:
        return current_demand
    
    current_demand = get_time_step_demand(actual_demand, 37)
    capacity = get_capacity_by_server_generation_latency_sensitivity(fleet)

    unsatisfied_demand = {}
    relevant_server_generations = np.union1d(current_demand.index.unique(), capacity.index.unique())
    for server_generation in relevant_server_generations:
        unsatisfied_demand[server_generation] = {}
        for latency_sensitivity in np.array(['low', 'medium', 'high']):

            if server_generation in current_demand.index.unique():
                this_demand = current_demand.loc[server_generation][latency_sensitivity]
            else:
                this_demand = 0

            if server_generation in capacity.index.unique():
                this_capacity = capacity.loc[server_generation][latency_sensitivity]
            else:
                this_capacity = 0

            unsatisfied_demand[server_generation][latency_sensitivity] = this_demand - this_capacity
        
    unsatisfied_demand = pd.DataFrame(unsatisfied_demand).transpose().rename_axis('server_generation')

    return unsatisfied_demand

unsatisfied_demand = get_unsatisfied_demand(actual_demand, state.fleet, 37)
print(unsatisfied_demand)

servers_to_buy = {}
for server_generation in unsatisfied_demand.index.unique():
    servers_to_buy[server_generation] = {}
    for latency_sensitivty in unsatisfied_demand.columns.unique():
        selling_price = selling_prices\
            .set_index(['server_generation', 'latency_sensitivity'])\
            .loc[server_generation, latency_sensitivty]['selling_price']

        d = unsatisfied_demand.loc[server_generation][latency_sensitivty]

        servers_to_buy[server_generation][latency_sensitivty] = round(d / selling_price)

        # print(f"{server_generation}: {latency_sensitivty}: selling_price={selling_price}, demand={d}")
        # print(f"Number of servers to buy = {d / selling_price}")
servers_to_buy = pd.DataFrame(servers_to_buy).transpose().rename_axis('server_generation')
print(servers_to_buy)


                      low  medium    high
server_generation                        
CPU.S1             382541  151514  365152
CPU.S2              14819    1234    9092
GPU.S1               1108     169    3535
                     low  medium   high
server_generation                      
CPU.S1             38254   10101  14606
CPU.S2              1482      82    364
GPU.S1                 1       0      2
